In [ ]:
from sqlalchemy import create_engine
from datetime import datetime
from sqlalchemy.sql.expression import null

import flask
from IPython import get_ipython

Conexão REST API

In [ ]:
!pip install flask

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-11-17 02:22:23--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.19M  59.7MB/s    in 0.2s    

2022-11-17 02:22:24 (59.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('python app.py &')
get_ipython().system_raw('./ngrok http 5000 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://911e-35-231-235-52.ngrok.io


In [ ]:
from flask import Flask
app = Flask(__name__)


@app.route('/')
def index():
    return "Welcome to Golab REST API"

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [17/Nov/2022 02:22:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2022 02:22:45] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2022 02:22:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2022 02:22:46] "GET /favicon.ico HTTP/1.1" 404 -


Craição do objeto contato e conexões

In [ ]:
class Contato:
    def __init__(self, rowid: int, nome: str, idade: int, datacadastro: str):
      self.rowid = int(rowid)
      self.nome = nome
      self.idade = int(idade)
      self.datacadastro = datacadastro
    
    def __str__(self):
      return f"{self.rowid} |{self.nome} |{self.idade} |{self.datacadastro}"

In [ ]:
#Cria a conexão com o database
my_conn = create_engine("sqlite:////content/database.db")

In [ ]:
#Create - INSERT
def CRUDcreate(Contato: Contato):
  
  #deleção da rowid desnessessaria
  del(Contato.rowid)
  
  auxNome = Contato.nome
  auxIdade = str(Contato.idade)
  auxDatacadastro = Contato.datacadastro

  my_conn.execute("INSERT INTO contatos VALUES ('"+auxNome+"', "+auxIdade+", '"+auxDatacadastro+"')")

In [ ]:
# Return - SELECT
def CRUDreturn(operacaoExtra = False):
  r_set=my_conn.execute("SELECT rowid, nome, idade, datacadastro FROM contatos")

  contatos = []

  #Armazena os Rowid que existem na tabela
  rowidExistente = []
  for row in r_set:
    auxRowid = row[0]
    rowidExistente.append(auxRowid)

    auxNome = row[1]
    auxIdade = row[2]
    auxDatacadastro = row[3]
    contatos.append(Contato(auxRowid, auxNome, auxIdade, auxDatacadastro))
    #print(Contato(auxRowid, auxNome, auxIdade, auxDatacadastro))

  #Variavel operacaoExtra criada
  #Se a operação pedida for Return imprime os retornos
  #Se a operação pedida for Update utilizamos a variavel auxRowid internamente
  if(operacaoExtra == False):
    for contato in contatos:
      print(contato)
  else:
    return rowidExistente

In [ ]:
#Update - UPDATE
def CRUDupdate(Contato: Contato, rowid: str):
  
  #deletar variavel desnessessaria
  del(Contato.rowid)

  auxNome = Contato.nome
  auxIdade = str(Contato.idade)
  auxDatacadastro = Contato.datacadastro

  rowidExistente = CRUDreturn(operacaoExtra=True)
  temRowid = False
  for rowidE in rowidExistente:
    if(int(rowid) == int(rowidE)):
      temRowid = True
      break

  if(temRowid == True):
    my_conn.execute("UPDATE contatos SET nome= '"+auxNome
          +"', idade= "+auxIdade
          +", datacadastro= '"+auxDatacadastro
          +"' WHERE rowid= "+ rowid)
  else:
    print("Rowid não encontrado")

In [ ]:
#Delete - DELETE
def CRUDdelete(rowid: int):
  rowidExistente = CRUDreturn(operacaoExtra=True)
  temRowid = False
  for rowidE in rowidExistente:
    if(int(rowid) == int(rowidE)):
      temRowid = True
      break
  
  if(temRowid == True):
    my_conn.execute("DELETE FROM contatos WHERE rowid= "+str(rowid))
  else:
    print("Rowid não encontrado")

In [ ]:
def criarContato():

  nome = str(input("Nome: "))
  idade = int(input("Idade: "))
  datacadastro = str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))

  #valor rowid usado para criação dos contatos
  #deletado internamente nas funções
  rowidTemp = 0

  contato = Contato(rowidTemp, nome, idade, datacadastro)

  return contato

Main

In [ ]:
operacao = input("Insira a operação desejada C,R,U ou D: ")

if(operacao == 'c'):
  CRUDcreate(criarContato())

else:
  if(operacao == 'r'):
     CRUDreturn()

  else:
    if(operacao == 'u'):
      rowid = int(input("Insira o rowid a ser atualizado: "))
      CRUDupdate(criarContato(), str(rowid))

    else:
      if(operacao == 'd'):
        rowid = int(input("Insira o rowid a ser deletado: "))
        CRUDdelete(rowid)

      else:
        print("Por favor insira a operação entre C, R, U ou D")


Insira a operação desejada C,R,U ou D: r
4 |Joaquina Josefina |19 |2022-11-17 01:45:07
5 |Siclano Delclanino |36 |2022-03-27 11:00:00
9 |Ismael |16 |2022-11-17 01:26:53
10 |Frederico |82 |2022-11-17 01:52:27
